In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from tqdm import tqdm

import time
import requests
import json

In [2]:
jobLinks = json.load(open('Data/jobLinks.json'))
len(jobLinks)

90

In [1]:
webdriver = webdriver.Chrome()

NameError: name 'webdriver' is not defined

In [28]:
# Job Title, Job Salary, Job Experience, Education, Location and Arrangement, Skills, Company & hirer name, Company links, Category, Company location
def getJobDetail(link):
    webdriver.get(link)
    wait = WebDriverWait(webdriver, 10)
    job = dict()

    jobHeader = wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, '//div[@class="TopFoldsc__JobOverviewContainer-sc-f7gyiq-2 fTcdeM"]//p')))
    skills = wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, '//div[@class="Skillssc__TagContainer-sc-gaj5e9-1 kwxkCn"]//span/div/p')))
    requirements = wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, '//div[@class="JobRequirementssc__TagsWrapper-sc-10j2bdt-1 ewBmwf"]//span/div/p')))
    hirer = wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, '//div[@class="HiringManagerSectionsc__CreatorInfoDetail-sc-ql38za-10 kJFJVW"]/div[1]/p')))
    company = wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, '//div[@class="HiringManagerSectionsc__CreatorInfoDetail-sc-ql38za-10 kJFJVW"]/div[2]/p/div')))
    companyLink = wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, '//div[@class="AboutCompanySectionsc__Main-sc-1cpbaxt-0 klxtwC"]/a')))
    category = wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, '//div[@class="JobCategoriessc__TagsWrapper-sc-1btru5u-1 bpLuhz"]//button//a')))
    location = wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, '//div[@class="AboutCompanySectionsc__Main-sc-1cpbaxt-0 klxtwC"]/div[3]/p[2]')))
    # Job Link
    job["Link"] = link
    # Job Title
    job["Title"] = webdriver.find_elements(By.XPATH, '//div[@class="TopFoldsc__JobOverviewContainer-sc-f7gyiq-2 fTcdeM"]//p[1]')[0].text
    # Job Salary
    job["Salary"] = [i.text for i in jobHeader[1].find_elements(By.TAG_NAME, 'span')]
    # # Job Experience 
    # job["Experience"] = jobHeader[2].text
    # # Education
    # job["Education"] = jobHeader[3].text
    # Location and Arrangement
    job["Location"] = webdriver.find_elements(By.XPATH, '//div[@aria-label="Job Location and Arrangement"]/p[1]/div[2]')[0].text
    # Time
    job["Last_updated"] = jobHeader[-1].text 
    # CrawlTime
    job["Created_time"] = time.strftime("%d-%m-%Y %H:%M:%S", time.localtime())
    # Skills
    job["Skills"] = [skill.text for skill in skills if skill.text != ""]
    # Job Requirement
    job["Requirement"] = [requirement.text for requirement in requirements if requirement.text != ""]
    # Company & hirer name
    job["Hirer"] = hirer[0].text
    job["Company"] = company[0].text
    # Company links
    job["Company_link"] = companyLink[0].get_attribute("href")
    # Company location
    job["Company_location"] = webdriver.find_elements(By.XPATH, '//div[@class="AboutCompanySectionsc__Main-sc-1cpbaxt-0 klxtwC"]/div[3]/p[2]')[0].text
    # Category
    job["Category"] = [category.text for category in category if category.text != ""]
    
    return job

In [25]:
webdriver.find_elements(By.XPATH, '//div[@class="AboutCompanySectionsc__Main-sc-1cpbaxt-0 klxtwC"]/div[3]/p[2]')[0].text

'Tầng 15 - 45A Lý Tự Trọng phường Bến Nghé quận 1 Thành phố Hồ Chí Minh, 18E Cộng Hòa phường 4 quận Tân Bình'

In [27]:
display(getJobDetail(jobLinks[9]))

{'Link': 'https://glints.com/vn/opportunities/jobs/aia-exchange-quan-1-truong-phong-kinh-doanh/8a38d33b-6f8e-435c-961b-cbb5a60a6c93?utm_referrer=explore&traceInfo=c367a96c60373bff80c4f095afb02436',
 'Title': '[AIA EXCHANGE - QUẬN 1] Trưởng phòng kinh doanh',
 'Salary': ['₫ 18-40 Tr/Tháng'],
 'Location': 'Quận Một, Thành phố Hồ Chí Minh  ·  Làm tại công ty  ·  Việc làm fulltime',
 'Last_updated': 'Cập nhật Hôm qua',
 'Created_time': '29-08-2024 22:39:34',
 'Skills': ['Teamwork', 'Strong Communication Skills', 'Microsoft Office'],
 'Requirement': ['1 - 3 năm kinh nghiệm', 'Tối thiểu Cử Nhân'],
 'Hirer': 'Ngọc Nguyễn',
 'Company': 'Nhà tuyển dụng · Công ty TNHH Bảo hiểm Nhân thọ AIA (Việt Nam)',
 'Company_link': 'https://glints.com/vn/companies/cong-ty-tnhh-bao-hiem-nhan-tho-aia-viet-nam/ec5438f0-025a-4fca-8b43-c8aa0da054d5',
 'Company_location': 'Tầng 15 - 45A Lý Tự Trọng phường Bến Nghé quận 1 Thành phố Hồ Chí Minh, 18E Cộng Hòa phường 4 quận Tân Bình',
 'Category': ['Management', 'Quản

In [29]:
import random
counter = 0
path = "Data/JobDetail"
fail = [] # save links that fail
for link in tqdm(jobLinks):
    try:
        job = getJobDetail(link)
        json.dump(job, open(path + "/job_" + str(counter) + ".json", "w", encoding="utf-8"), indent=4, ensure_ascii=False)
        counter += 1
    except:
        fail.append(link)
    finally:
        time.sleep(random.randint(3, 5))
        continue

100%|██████████| 90/90 [12:29<00:00,  8.32s/it]


In [ ]:
webdriver.close()